### K-Nearest Neighbor(KNN) - Classificação

Algorítimo de classificação baseado na quantidade majoritária de visinhos mais próximos.

### Conjunto de dados de sobrevivência de Haberman
https://archive.ics.uci.edu/ml/datasets/Haberman%27s+Survival

Conjunto de dados contendo casos de estudo realizado sobre a sobrevivência de pacientes que se submeteram a cirurgia para câncer de mama 

In [1]:
# leitura do arquivo haberman
amostras = []

with open('haberman.data', 'r') as f:
    for linha in f.readlines():
        atributos = linha.replace('\n', '').split(',');

        amostras.append([
            int(atributos[0]),  # Idade do paciente no momento da operação
            int(atributos[1]),  # Ano da operação
            int(atributos[2]),  # Número de nódulos detectados
            int(atributos[3])   # (1-Sobreviveu 5 anos ou mais; 2-Feleceu dentro de 5 anos)
        ])

print(f'Quantidade de amostras - {len(amostras)}')
print(f'Primeira amostra - {amostras[0]}')

Quantidade de amostras - 306
Primeira amostra - [30, 64, 1, 1]


In [2]:
# Contando quantidade de amostras 1 ou 2
from collections import Counter

sobreviveu = Counter([registro[3] for registro in amostras])

print(f'Quant que pacientes que sobreviveram 5 ou mais anos - {sobreviveu[1]}')
print(f'Quant que pacientes que faleceram em até 5 anos     - {sobreviveu[2]}')

Quant que pacientes que sobreviveram 5 ou mais anos - 225
Quant que pacientes que faleceram em até 5 anos     - 81


In [3]:
# Separando amostras
pacientes_sobrev = [registro for registro in amostras if registro[3] == 1]
pacientes_falece = [registro for registro in amostras if registro[3] == 2]

In [4]:
# Dividindo dados entre 60% para treino e 40% para teste
treino, teste = [], []

treino.extend(pacientes_sobrev[0 : int(len(pacientes_sobrev) * 0.6)])
treino.extend(pacientes_falece[0 : int(len(pacientes_falece) * 0.6)])

teste.extend(pacientes_sobrev[int(len(pacientes_sobrev) * 0.6) :])
teste.extend(pacientes_falece[int(len(pacientes_falece) * 0.6) :])

print(f'{len(treino)} para treino, {len(teste)} para teste')

183 para treino, 123 para teste


### Distância Euclidiana para determinar o vizinho mais próximo

In [22]:
import math

def dis_euclidiana(v1, v2):
    dim, soma = len(v1), 0
    
    for i in range(dim - 1):
        soma += math.pow(v1[i] - v2[i], 2)
    
    return math.sqrt(soma)

# teste da função euclidiana
v1 = [1, 2, 3]
v2 = [2, 1, 5]
dis_euclidiana(v1, v2) # 1.4142135623730951

1.4142135623730951

In [6]:
# Implementação da função KNN
def knn(treinamento, nova_amostra, k):
    dists = {}
    
    # Calcular a distância da nova amostra para todos
    # as amostras de treinamento
    for i in range(len(treinamento)):
        d = dis_euclidiana(treinamento[i], nova_amostra)
        dists[i] = d
    
    # Obtém indices dos k-vizinhos mais próximos
    k_vizinhos = sorted(dists, key=dists.get)[:k]
    
    # Contágem de 'votos'
    votacao = Counter([treinamento[indice][3] for indice in k_vizinhos])
    
    # Retorna o majoritário
    return votacao.most_common(1)[0]

In [7]:
# Avaliação com uma só amostra de teste
print(teste[0])

vencedor, quantidade_votos = knn(treino, teste[0], k=13)
print(f'Vencedor {vencedor} com {quantidade_votos} votos')

[55, 66, 0, 1]
Vencedor 1 com 11 votos


In [8]:
# Avaliação com todos os testes e acurácia
acertos = 0

for i in range(len(teste)):
    vencedor, quantidade_votos = knn(treino, teste[i], k=11)
    acertos += 1 if teste[i][3] == vencedor else 0
    # print(f'Correto {teste[i][3]} - Vencedor {vencedor} com {quantidade_votos} votos')
    
print('Acurácia de %.2f%%' % (acertos/len(teste)*100))

Acurácia de 77.24%


## Implementação do KNN com Scikit-Learn
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier

In [9]:
from sklearn.neighbors import KNeighborsClassifier

# p=1 - Distância Manhatan
# p=2 - Distância Euclidiana *default
knn = KNeighborsClassifier(n_neighbors=11, p=2)

In [10]:
# Separando dados de treino e teste
# entradas- idade e número de nódulos
# saidas  - 1-Sobreviveu 5 anos ou mais; 2-Feleceu dentro de 5 anos
entradas_treino = [[registro[0], registro[2]] for registro in treino]
saidas_treino   = [registro[3] for registro in treino]

entradas_teste  = [[registro[0], registro[2]] for registro in teste]
saidas_teste    = [registro[3] for registro in teste]

In [11]:
# Treinando o knn
knn.fit(entradas_treino, saidas_treino)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='uniform')

In [12]:
# Testando o knn
previsoes = knn.predict(entradas_teste)

In [13]:
# Avaliação com todos os testes e acurácia
acertos = 0

for i in range(len(entradas_teste)):
    acertos += 1 if previsoes[i] == saidas_teste[i] else 0
    
print('Acurácia de %.2f%%' % (acertos/len(teste)*100))

Acurácia de 74.80%


## Implementação do kNN com Numpy e sklearn

### Utilizando o Balance Scale Data Set
Este conjunto de dados foi gerado para modelar resultados experimentais psicológicos. Cada exemplo é classificado como tendo a escala de equilíbrio inclinada para a direita, ponta para a esquerda ou balanceada. Os atributos são o peso esquerdo, a distância esquerda, o peso certo e a distância certa. A maneira correta de encontrar a classe é a maior entre (distância à esquerda * peso à esquerda) e (distância à direita * peso à direita). Se eles forem iguais, está equilibrado. 

#### Attribute Information:

1. Class Name: 3 (L, B, R)
2. Left-Weight: 5 (1, 2, 3, 4, 5)
3. Left-Distance: 5 (1, 2, 3, 4, 5)
4. Right-Weight: 5 (1, 2, 3, 4, 5)
5. Right-Distance: 5 (1, 2, 3, 4, 5)

625 amostras

In [14]:
# leitura do arquivo Balance Scale Data Set
entradas = []
saidas   = []

with open('balance-scale.data', 'r') as f:
    for linha in f.readlines():
        atributos = linha.replace('\n', '').split(',');

        entradas.append([
            int(atributos[1]),  # peso esquerdo
            int(atributos[2]),  # distância esquerda
            int(atributos[3]),  # peso direto
            int(atributos[4])   # distância direto
        ])
        
        if   atributos[0] == 'L': # L-esquerda
            saidas.append(1)
        elif atributos[0] == 'B': # B-balanceado
            saidas.append(2)
        elif atributos[0] == 'R': # R-direita
            saidas.append(3)

print(f'Quantidade de amostras - {len(entradas)}')
print(f'Primeira amostra - {entradas[0]} - {saidas[0]}')

Quantidade de amostras - 625
Primeira amostra - [1, 1, 1, 1] - 2


In [15]:
# Convertendo para array numpy
import numpy as np

X = np.array(entradas)
y = np.array(saidas)

In [16]:
# Dividindo em dados de treino e teste com train_test_split
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train_test_split#sklearn.model_selection.train_test_split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f'Quantidade de dados para treino - {len(X_train)}')
print(f'Quantidade de dados para teste  - {len(X_test)}')

Quantidade de dados para treino - 437
Quantidade de dados para teste  - 188


In [17]:
# Configurando e treinando o knn
knn = KNeighborsClassifier(n_neighbors=17, p=2)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=17, p=2,
                     weights='uniform')

In [18]:
# Executando o previsão
previsao = knn.predict(X_test)

#### Modos de Avaliação de Acurária

In [19]:
# Avaliando a acurácia com metrics
from sklearn import metrics

acertos_knn = metrics.accuracy_score(y_test, previsao)

print('Acurácia de %.2f%%' % (acertos_knn*100))

Acurácia de 86.70%


In [28]:
# Avaliando a acurácia com sum do numpy

print('Acurácia de %.2f%%' % (np.sum(y_test == previsao) / len(y_test) *100))
print('Acurácia de %.2f%%' % ((y_test == previsao).sum() / len(y_test) *100))

Acurácia de 86.70%
Acurácia de 86.70%


In [31]:
# Avaliando a acurácia com mean do numpy

print('Acurácia de %.2f%%' % (np.mean(y_test == previsao)*100))

Acurácia de 86.70%


In [32]:
# Avaliação do score pelo próprio knn
print('Acurácia de %.2f%%' % (knn.score(X_test, y_test)*100))

Acurácia de 86.70%


### Arquivo Senior/Fourth
Dado o tamanho do calsado e altura, decidir se é sênio ou fourth

In [34]:
import pandas as pd 

train_senior_fourth = pd.read_csv('train_senior_fourth.csv')
test_senior_fourth  = pd.read_csv('test_senior_fourth.csv')

In [48]:
test_senior_fourth.head()

,shoe size,height,class
0,12.985509,66.632266,seniors
1,9.957683,70.057887,seniors
2,10.366656,67.945519,seniors
3,10.484072,74.846184,seniors
4,10.273374,66.690172,seniors


In [85]:
# Separando Entradas e Saidas, transformando pandas p/ numpy
col_entradas = ['shoe size', 'height']
col_saida    = ['class']

x_train_senior_fourth  = train_senior_fourth[col_entradas].to_numpy()
y_train_senior_fourth  = train_senior_fourth[col_saida].to_numpy()

x_test_senior_fourth   = test_senior_fourth[col_entradas].to_numpy()
y_test_senior_fourth   = test_senior_fourth[col_saida].to_numpy()

In [86]:
# Confirgurando o knn
# Parâmetro weights, 
#    - uniforme, *padrão, todos os pontos são ponderados igualmente
#    - distance, pontos mais próximos terão uma influencia mais que os mais distantes
knn = KNeighborsClassifier(n_neighbors=3, weights='distance')

knn.fit(x_train_senior_fourth, y_train_senior_fourth.ravel())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')

In [87]:
previsao = knn.predict(x_test_senior_fourth)

acertos_knn = metrics.accuracy_score(y_test_senior_fourth, previsao)

print('Acurácia de %.2f%%' % (acertos_knn*100))

Acurácia de 99.33%
